In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from models.CR import CR3D
from models.CRJ import CRJ3D
from models.MCI import MCI3D
from models.ESN import ESN3D
from models.SAR import SAR3D
from models.SparseESN import SparseESN3D
from models.SW import SW3DSegregated, SW3DRandom
from models.HFR import HFRRes3D
from models.MC import MicrocolumnRes3D
from models.SwirlGatedMultiCycle import MPPRN
from models.DWMSR import DWMSR3D
from metrics.metrics import mse_dimwise, nrmse_dimwise, compute_valid_prediction_time, compute_attractor_deviation, compute_relative_psd

In [3]:
def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x*(rho - z) - y
    dzdt = x*y - beta*z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    num_steps = int(tmax / dt) + 1 # +1 to include t=0
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

In [4]:
def evaluate_nrmse(all_preds, test_target, horizons):
    """
    Evaluate model performance over multiple prediction horizons
    for teacher-forced single-step forecasting or autoregressive rollout.
    """
    horizon_nrmse = {}
    for horizon in horizons:
        preds = all_preds[:horizon]
        targets = test_target[:horizon]
        squared_errors = (preds - targets) ** 2
        variance = np.var(targets, axis=0)
        variance[variance == 0] = 1e-8  # avoid divide-by-zero
        nrmse = np.sqrt(np.sum(squared_errors) / (horizon * np.sum(variance)))
        horizon_nrmse[horizon] = nrmse
    return horizon_nrmse

In [5]:
def compute_valid_prediction_time(y_true, y_pred, t_vals, threshold, lambda_max, dt):
    """
    Compute the Valid Prediction Time (VPT) and compare it to Lyapunov time T_lambda = 1 / lambda_max.
    
    Parameters
    ----------
    y_true : ndarray of shape (N, dim)
        True trajectory over time.
    y_pred : ndarray of shape (N, dim)
        Model's predicted trajectory over time (closed-loop).
    t_vals : ndarray of shape (N,)
        Time values corresponding to the trajectory steps.
    threshold : float, optional
        The error threshold, default is 0.4 as in your snippet.
    lambda_max : float, optional
        Largest Lyapunov exponent. Default=0.9 for Lorenz.
        
    Returns
    -------
    T_VPT : float
        Valid prediction time. The earliest time at which normalized error surpasses threshold
        (or the last time if never surpassed).
    T_lambda : float
        Lyapunov time = 1 / lambda_max
    ratio : float
        How many Lyapunov times the model prediction remains valid, i.e. T_VPT / T_lambda.
    """
    # 1) Average of y_true
    y_mean = np.mean(y_true, axis=0)  # shape (dim,)
    
    # 2) Time-averaged norm^2 of (y_true - y_mean)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))  # scalar
    
    # 3) Compute the normalized error delta_gamma(t) = ||y_true - y_pred||^2 / denom
    diff = y_true - y_pred
    err_sq = np.sum(diff**2, axis=1)  # shape (N,)
    delta_gamma = err_sq / denom      # shape (N,)
    
    # 4) Find the first time index where delta_gamma(t) exceeds threshold
    idx_exceed = np.where(delta_gamma > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = t_vals[-1]
    else:
        T_VPT = t_vals[idx_exceed[0]]
    
    # 5) Compute T_lambda and ratio
    T_lambda = 1.0 / lambda_max

    # print(f"\n--- Valid Prediction Time (VPT) with threshold={threshold}, lambda_max={lambda_max} ---")

    T_VPT = (T_VPT - t_vals[0])  # Adjust T_VPT to be relative to the start time
    ratio = T_VPT / T_lambda

    return T_VPT, T_lambda, ratio

In [6]:
def compute_attractor_deviation(predictions, targets, cube_size=(0.1, 0.1, 0.1)):
    """
    Compute the Attractor Deviation (ADev) metric.

    Parameters:
        predictions (numpy.ndarray): Predicted trajectories of shape (n, 3).
        targets (numpy.ndarray): True trajectories of shape (n, 3).
        cube_size (tuple): Dimensions of the cube (dx, dy, dz).

    Returns:
        float: The ADev metric.
    """
    # Define the cube grid based on the range of the data and cube size
    min_coords = np.min(np.vstack((predictions, targets)), axis=0)
    max_coords = np.max(np.vstack((predictions, targets)), axis=0)

    # Create a grid of cubes
    grid_shape = ((max_coords - min_coords) / cube_size).astype(int) + 1

    # Initialize the cube occupancy arrays
    pred_cubes = np.zeros(grid_shape, dtype=int)
    target_cubes = np.zeros(grid_shape, dtype=int)

    # Map trajectories to cubes
    pred_indices = ((predictions - min_coords) / cube_size).astype(int)
    target_indices = ((targets - min_coords) / cube_size).astype(int)

    # Mark cubes visited by predictions and targets
    for idx in pred_indices:
        pred_cubes[tuple(idx)] = 1
    for idx in target_indices:
        target_cubes[tuple(idx)] = 1

    # Compute the ADev metric
    adev = np.sum(np.abs(pred_cubes - target_cubes))
    return adev

In [7]:
initial_state  = [1.0, 1.0, 1.0]
tmax = 250
dt = 0.02
t_vals, lorenz_traj = generate_lorenz_data(
    initial_state=initial_state,
    tmax=tmax,
    dt=dt
)

washout = 2000
t_vals = t_vals[washout:]
lorenz_traj = lorenz_traj[washout:]

scaler = MinMaxScaler()
scaler.fit(lorenz_traj)
lorenz_traj = scaler.transform(lorenz_traj)

T_data = len(lorenz_traj)
train_frac = 0.8
train_end = int(train_frac * (T_data - 1))
train_input = lorenz_traj[:train_end]
train_target = lorenz_traj[1:train_end + 1]
test_input = lorenz_traj[train_end:-1]
test_target = lorenz_traj[train_end + 1:]
n_test_steps = len(test_input)
initial_in = test_input[0]

T_data = len(lorenz_traj)
print(f"Data length: {T_data}, from t=0..{tmax} with dt={dt}.")

Data length: 10501, from t=0..250 with dt=0.02.


In [8]:
# sph_res = MPPRN(
#     reservoir_size        = 300,      # total neurons  (must be multiple of 3)
#     cycle_weight          = 0.97,      # r  weight on each edge of the three base cycles
#     bridge_weight         = 0.2,     # s  weight on the three inter-cycle “bridge” edges
#     input_scale           = 0.7,      # |W_in| multiplier   (was 0.5 by default)
#     leak_rate             = 0.8,      # α  leaky-integrator coefficient (1.0 ⇒ no leak)
#     ridge_alpha           = 1e-6,     #  penalty for the read-out ridge regression
#     swirl_beta            = 12.5,      # β  controls the steepness of the swirl gate
#     swirl_frequency       = None,     # ω  spatial frequency (defaults to 2π / (N/3))
#     swirl_sigmoid         = True,     # if False, uses raw sin; if True, uses σ(β sin(…))
#     seed                  = 42,       # seed 
#     use_polynomial_readout= True      # augment read-out with square andd bias
#     )
# sph_res.fit_readout(train_input, train_target, discard=1000)
# sph_preds = sph_res.predict_autoregressive(initial_input=initial_in, n_steps=n_test_steps)

In [16]:
# Unweighted Erdős–Rényi random graph
from scipy import sparse

n0      = 300               # number of nodes
p_edge  = 4 / n0            # expected degree ≈ 4

rng  = np.random.default_rng(123)
rows = rng.choice(n0, size=int(p_edge * n0 * (n0 - 1) // 2))
cols = rng.choice(n0, size=rows.size)
mask = rows != cols         # avoid self-loops
rows, cols = rows[mask], cols[mask]


# build upper triangle, then symmetrise
adj = sparse.csr_matrix((np.ones_like(rows), (rows, cols)), shape=(n0, n0))
adj = adj + adj.T
adj[adj > 0] = 1.0          # make unweighted (0/1)

In [ ]:
sph_res = DWMSR3D(
        adj              = adj,                 # (required) sparse CSR adjacency
        num_scales       = 2,                   # S – number of coarse levels
        tau0             = 0.01,                # base diffusion time τ₀
        betas            = [0.5, 0.4],          # funnel strengths β₁…β_S
        alphas           = [0.6, 0.7, 1.0],     # leak rates α₀…α_S  (len = S+1)
        input_scale      = 0.1,                 # scaling of random W_in
        ridge_alpha      = 1e-4,                # ℓ₂ penalty in ridge read-out
        detail_features  = True,                # include Δ_s features?
        seed             = 123                  # RNG seed for W_in & warnings
    )
sph_res.fit_readout(train_input, train_target, discard=1000)
sph_preds = sph_res.predict_autoregressive(initial_input=initial_in, n_steps=n_test_steps)

TypeError: DWMSR3D.__init__() got an unexpected keyword argument 'frequency_mode'

In [21]:
_, _, sph_ratio = compute_valid_prediction_time(
    y_true=test_target,
    y_pred=sph_preds,
    t_vals=t_vals,
    threshold=0.4,
    lambda_max=0.9,
    dt=dt
)

In [22]:
sph_ratio

np.float64(2.4660000000000015)

In [13]:
sph_nrmse = evaluate_nrmse(sph_preds, test_target, horizons=[1000])
print(f"MPPRN NRMSE: {sph_nrmse}")

MPPRN NRMSE: {1000: np.float64(1.3461871583353822)}
